In [2]:
import sys
sys.path.append('../')
import dfm
import fm
import lr

In [3]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
import torch.fx as fx
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [24]:
l =[    241,       8,       8,    3697,    4614,      25,    5481,
        329,      31,  381763, 1611748,    6793,       6,       5,
      2509,       9,      10,     432,       5,      68,     169,
        61]

In [3]:
def gen_pattern_replace_and_matcher_for_LR(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.fc = torch.nn.Embedding(1, 1)
      self.bias1 = torch.nn.Parameter(torch.zeros((1,)))
      self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))

    def forward(self,x):
      x = x + self.offsets
      return torch.sum(self.fc(x), dim=1) + self.bias1 
  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[target_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  fc_node = node_map[pattern_env['fc']]
  fc_node_module = utils.get_target_mod(traced,fc_node.target)
  
  bias_node = node_map[pattern_env['bias1']]
  bias_val = utils.get_target_mod(traced,bias_node.target)
  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      embedding_config = fc_node_module.weight.data.shape
      self.fc = nn.Embedding(embedding_config[0],embedding_config[1])
      # self.fc = fc_node_module
      self.fc.weight.data.copy_(fc_node_module.weight.data)
      self.bias = bias_val

    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_sum = torch.sum(self.fc(redency_part)) + self.bias
      unredency_sum = torch.sum(self.fc(unredency_part),dim=1)
      return redency_sum + unredency_sum
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [25]:
lr_model = lr.LogisticRegressionModel(l)
lr_model_traced = fx.symbolic_trace(lr_model)

In [34]:
interp = utils.ProfilingInterpreter(lr_model_traced)
interp.run(torch.randint(low=0, high=5, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 1.428365707397461 ms
Op type        Op                Average runtime (s)    Pct total runtime
-------------  --------------  ---------------------  -------------------
call_module    linear_fc                 0.000481367             33.7006
call_function  add                       0.000336885             23.5854
call_function  sum_1                     0.000117064              8.19563
call_function  sigmoid                   5.60284e-05              3.92255
call_function  add_1                     5.10216e-05              3.57202
call_method    squeeze                   4.60148e-05              3.2215
placeholder    x                         4.05312e-05              2.83759
get_attr       linear_bias               4.02927e-05              2.8209
get_attr       linear_offsets            3.83854e-05              2.68736
output         output                    2.28882e-05              1.6024


In [35]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_LR(lr_model_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "linear_offsets")

In [36]:
matches = subgraph_rewriter.replace_pattern_with_filters(lr_model_traced, pattern, replace,[match])

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [46]:
interp = utils.ProfilingInterpreter(lr_model_traced)
interp.run(torch.randint(low=0, high=5, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.0301342010498047 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    fc_1                        0.000469446             23.1239
call_function  add_1                       0.000273228             13.4586
call_module    fc                          0.000148773              7.32824
call_function  sum_2                       0.000119209              5.87199
call_function  getitem                     9.84669e-05              4.85026
call_function  sigmoid                     7.03335e-05              3.46447
call_function  sum_1                       6.53267e-05              3.21785
call_function  add_3                       5.79357e-05              2.85379
call_function  add                         5.72205e-05              2.81856
call_function  getitem_1                   5.67436e-05              2.79507
call_method    squeeze                     5.62668e-05  

In [90]:
matches

[ReplacedPatterns(anchor=add_1, nodes_map={add_1: add_1, sum_1: sum_1, fc: linear_fc, add: add, x: x, offsets: linear_offsets, bias1: linear_bias}, replacements=[add_2])]

In [4]:
def gen_pattern_replace_and_matcher_for_FM(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  target_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)
      self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))

    def forward(self,x):
      x = x + self.offsets
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_embed = self.embed(redency_part)
      unredency_embed = self.embed(unredency_part)
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      redency_embed_square_sum = torch.sum(redency_embed ** 2,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_embed ** 2,dim=1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [5]:
fm_model = fm.FactorizationMachineModel([100 for i in range(100)],64)
fm_model_traced = fx.symbolic_trace(fm_model)

In [7]:
interp = utils.ProfilingInterpreter(fm_model_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 24.988889694213867 ms
total time: 37.81890869140625 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    embedding_embedding               8.30436             21.9582
call_function  pow_2                             5.21374             13.7861
call_function  add                               2.69175              7.11746
call_function  sum_3                             2.12336              5.61454
call_function  sum_2                             1.94502              5.14298
call_function  sum_1                             1.34325              3.5518
call_module    linear_fc                         1.01805              2.6919
call_function  sub                               0.631809             1.67062
call_function  add_2                             0.564814             1.49347
call_function  pow_1                             0.436306             1

# Test if LR pattern works for FM model

In [70]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_LR(fm_model_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "linear_offsets")

In [71]:
matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced, pattern, replace,[match])

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset_1 target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  

In [76]:
interp = utils.ProfilingInterpreter(fm_model_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 3.1082630157470703 ms
Op type        Op                    Average runtime (s)    Pct total runtime
-------------  ------------------  ---------------------  -------------------
call_module    fc_1                          0.00031352             10.0867
call_function  add_1                         0.000253439             8.15372
call_function  pow_3                         0.000214577             6.90343
call_module    embed_1                       0.000207663             6.68098
call_function  sum_4                         0.000197411             6.35115
call_function  sum_2                         0.000190973             6.14405
call_function  add_5                         0.000131845             4.24177
call_function  add_3                         9.65595e-05             3.10654
call_function  add_2                         9.10759e-05             2.93012
call_function  sum_5                         8.84533e-05             2.84575
call_function  pow_1                     

# Test if FM pattern works for FM model

In [62]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_FM(fm_model_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "linear_offsets")

In [63]:
matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced, pattern, replace,[match])

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [69]:
interp = utils.ProfilingInterpreter(fm_model_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 3.2587051391601562 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    linear_fc                   0.000436306            13.3889
call_function  add                         0.000287533             8.82353
call_function  pow_3                       0.0002141               6.57009
call_module    embed_1                     0.000209332             6.42376
call_function  sum_5                       0.000194788             5.97747
call_function  sum_3                       0.000184059             5.64823
call_function  add_3                       0.000145435             4.46298
call_function  sum_1                       0.000139713             4.28739
call_function  add_5                       0.000101566             3.11677
call_function  add_4                       9.53674e-05             2.92654
call_module    embed                       9.32217e-05           

In [550]:
def workload_fm(dim):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(22)],dim)
  fm_model_traced_ori = fx.symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(22)],dim)
  fm_model_traced_modify = fx.symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM(fm_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

In [549]:
import time


# dim = 8

In [551]:
ori , modify = workload_fm(8)

now gen workload of fm with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [552]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.7771571477254232

In [553]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.8071184158325195

In [554]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.0961135228474934

In [555]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.2012084325154622

In [472]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

927 µs ± 110 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [475]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

961 µs ± 128 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [477]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.05 ms ± 171 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [480]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.11 ms ± 194 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [ ]:
interp = utils.ProfilingInterpreter(ori)


In [ ]:
%timeit -n 1 -r 300 interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

In [115]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 1.6362667083740234 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_module    embedding_embedding            0.000312805            19.117
call_module    linear_fc                      0.00022316             13.6384
call_function  pow_2                          0.000136375             8.33455
call_function  sum_3                          8.44002e-05             5.15809
call_function  sum_2                          8.24928e-05             5.04153
call_function  sum_1                          7.96318e-05             4.86668
call_function  add                            7.4625e-05              4.56069
call_function  add_2                          6.22272e-05             3.803
call_function  pow_1                          4.74453e-05             2.89961
call_function  mul                            4.72069e-05             2.88504
call_function  sum_4             

In [107]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 1.9772052764892578 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    embed_1                     0.00030899             15.6276
call_module    linear_fc                   0.000252962            12.7939
call_function  pow_3                       0.0001266               6.40299
call_function  add                         0.000116348             5.88448
call_function  sum_5                       6.98566e-05             3.5331
call_function  sum_3                       6.65188e-05             3.36428
call_function  sum_1                       5.19753e-05             2.62872
call_module    embed                       4.74453e-05             2.39961
get_attr       linear_offsets              4.673e-05               2.36344
placeholder    x                           4.57764e-05             2.31521
call_function  sum_6                       4.41074e-05             

In [125]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.4831295013427734 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_module    linear_fc                      0.000551939            22.2276
call_function  add                            0.000313282            12.6164
call_module    embedding_embedding            0.00020647              8.31493
call_function  sum_2                          0.000170231             6.8555
call_function  pow_2                          0.000146866             5.91455
call_function  add_2                          0.000145674             5.86654
call_function  sum_3                          0.000133038             5.35766
call_function  sum_1                          9.53674e-05             3.84061
call_function  pow_1                          7.4625e-05              3.00528
call_function  sum_4                          7.34329e-05             2.95727
call_function  sub             

In [138]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.7201175689697266 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    linear_fc                   0.000566483            20.8257
call_function  add                         0.000341892            12.569
call_module    embed_1                     0.000170231             6.25822
call_function  add_3                       0.000142574             5.24148
call_function  pow_3                       0.0001297               4.76817
call_function  sum_5                       0.000103235             3.79525
call_function  sum_1                       9.60827e-05             3.5323
call_function  sum_3                       9.60827e-05             3.5323
call_function  add_5                       8.63075e-05             3.17293
call_function  add_4                       7.98702e-05             2.93628
call_function  sum_6                       6.79493e-05             2.

# dim = 32

In [507]:
ori , modify = workload_fm(32)

now gen workload of fm with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [510]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))


1.07 ms ± 131 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [511]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

1.13 ms ± 132 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [512]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


2.07 ms ± 207 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [513]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.17 ms ± 235 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [156]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.465963363647461 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_module    embedding_embedding            0.000392675            15.9238
call_module    linear_fc                      0.000388384            15.7498
call_function  add                            0.000316381            12.8299
call_function  sum_2                          0.0001688               6.84521
call_function  pow_2                          0.000123978             5.02755
call_function  sum_3                          0.000117779             4.77618
call_function  sum_1                          8.08239e-05             3.27758
call_function  sum_4                          7.96318e-05             3.22924
call_function  pow_1                          7.20024e-05             2.91985
call_function  sub                            6.96182e-05             2.82317
call_function  add_2            

In [201]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.5565624237060547 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_function  add                         0.000489473            19.1458
call_module    embed_1                     0.000296831            11.6106
call_module    linear_fc                   0.000217199             8.49576
call_function  pow_3                       0.00014782              5.78196
call_function  sum_5                       0.00012064              4.71883
call_function  sum_3                       0.000110626             4.32715
call_function  add_5                       7.79629e-05             3.04952
call_function  sum_6                       6.86646e-05             2.68582
call_function  add_4                       6.84261e-05             2.67649
call_function  sum_1                       6.10352e-05             2.38739
call_function  pow_1                       5.50747e-05            

In [239]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 3.0117034912109375 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_module    linear_fc                      0.000631332            20.9626
call_function  add                            0.000365019            12.12
call_module    embedding_embedding            0.000234365             7.78182
call_function  pow_2                          0.000224113             7.44142
call_function  sum_2                          0.000188589             6.26187
call_function  sum_3                          0.000183344             6.08771
call_function  add_2                          0.000154495             5.12983
call_function  pow_1                          0.000119925             3.98195
call_function  sub                            0.00011301              3.75237
call_function  sum_1                          0.000111341             3.69696
call_function  sum_4            

In [233]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 3.078937530517578 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    linear_fc                   0.00066781             21.6896
call_function  add                         0.000402451            13.0711
call_function  sum_5                       0.000151634             4.92489
call_function  add_3                       0.000137568             4.46802
call_function  sum_3                       0.000137568             4.46802
call_module    embed_1                     0.000129223             4.197
call_function  pow_3                       0.000111341             3.61623
call_function  sum_1                       9.87053e-05             3.20582
call_function  sum_6                       8.58307e-05             2.78767
call_function  add_5                       8.32081e-05             2.70249
call_function  add_4                       8.24928e-05             2.

# 64

In [542]:
ori , modify = workload_fm(64)

now gen workload of fm with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [544]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))


1.19 ms ± 158 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [545]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

1.28 ms ± 174 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [547]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


2.22 ms ± 241 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [548]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.2 ms ± 261 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [267]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.44903564453125 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_function  add                            0.000329018            13.4346
call_module    embedding_embedding            0.000305176            12.4611
call_module    linear_fc                      0.000257015            10.4945
call_function  add_2                          0.000215769             8.81036
call_function  sum_2                          0.000181913             7.42796
call_function  pow_2                          0.000159502             6.51285
call_function  sum_3                          0.000153542             6.26947
call_function  sub                            8.72612e-05             3.56308
call_function  pow_1                          8.65459e-05             3.53388
call_function  sum_1                          8.32081e-05             3.39759
call_function  sum_4             

In [279]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.31170654296875 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    embed_1                     0.000279665            12.0978
call_function  sum_1                       0.000196457             8.49835
call_module    linear_fc                   0.000194788             8.42616
call_function  sum_5                       0.000148058             6.4047
call_function  sum_3                       0.00013876              6.00248
call_function  pow_3                       0.000135183             5.84777
call_function  pow_1                       8.46386e-05             3.6613
call_function  add_5                       8.01086e-05             3.46535
call_function  add_4                       7.79629e-05             3.37252
call_function  sum_6                       7.24792e-05             3.13531
call_function  sub                         6.79493e-05             2.

In [319]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 7.499456405639648 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_function  pow_2                          0.00244689             32.6276
call_module    embedding_embedding            0.00240111             32.0172
call_module    linear_fc                      0.000494719             6.59673
call_function  add                            0.000316858             4.22508
call_function  sum_3                          0.000289917             3.86584
call_function  sum_2                          0.000268221             3.57654
call_function  sub                            0.000159264             2.12367
call_function  pow_1                          0.000132322             1.76443
call_function  add_2                          0.000131369             1.75171
call_function  sum_1                          0.000105858             1.41154
call_function  sum_4           

In [314]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 3.613710403442383 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    linear_fc                   0.000612736            16.9559
call_function  add                         0.000353813             9.79086
call_function  pow_3                       0.000219345             6.0698
call_module    embed_1                     0.000201225             5.56838
call_function  sum_5                       0.000199556             5.5222
call_function  sum_3                       0.000188828             5.22531
call_function  add_3                       0.000144482             3.99815
call_function  add_4                       0.000137329             3.80022
call_function  sum_1                       0.000116348             3.21963
call_function  add_5                       0.000111103             3.07449
call_function  sum_6                       0.000110626             3

# dim = 128

In [532]:
ori , modify = workload_fm(128)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [540]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))


1.32 ms ± 158 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [541]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

1.27 ms ± 187 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [537]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


14 ms ± 1.15 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [538]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.34 ms ± 734 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [339]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.4564266204833984 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_module    embedding_embedding            0.000372171            15.1509
call_function  sum_2                          0.000351667            14.3162
call_function  pow_2                          0.000330925            13.4718
call_function  sum_3                          0.000216722             8.82267
call_module    linear_fc                      0.000178337             7.26002
call_function  pow_1                          0.000133753             5.44502
call_function  sub                            0.000132799             5.40619
call_function  sum_4                          9.01222e-05             3.66883
call_function  mul                            7.93934e-05             3.23207
call_function  add                            6.4373e-05              2.6206
call_function  add_2            

In [361]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 2.912759780883789 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    embed_1                     0.00033474             11.4922
call_module    linear_fc                   0.000292778            10.0516
call_function  add                         0.000198841             6.82655
call_function  sum_5                       0.000184774             6.34362
call_function  sum_3                       0.000163794             5.62331
call_function  pow_3                       0.000159502             5.47598
call_module    embed                       9.91821e-05             3.40509
call_function  add_5                       9.36985e-05             3.21683
call_function  sum_6                       9.01222e-05             3.09405
call_function  add_4                       8.7738e-05              3.0122
call_function  sub                         8.58307e-05             2

In [379]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 18.2955265045166 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_module    embedding_embedding            0.00466299            25.4871
call_function  pow_2                          0.00436425            23.8542
call_function  sum_3                          0.000947475            5.17873
call_module    linear_fc                      0.000658035            3.5967
call_function  sum_2                          0.000637054            3.48202
call_function  add                            0.000366449            2.00295
call_function  sub                            0.000215054            1.17544
call_function  pow_1                          0.000167131            0.91351
call_function  add_2                          0.000153542            0.83923
call_function  sum_4                          0.000147343            0.805348
call_function  sum_1                      

In [393]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 8.809089660644531 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_module    embed_1                     0.00267339             30.3481
call_function  pow_3                       0.00259995             29.5145
call_module    linear_fc                   0.000618458             7.02068
call_function  sum_5                       0.000402927             4.574
call_function  add                         0.000378132             4.29252
call_function  sum_3                       0.00028348              3.21804
call_function  add_3                       0.000249386             2.83101
call_function  add_5                       0.000150442             1.70781
call_function  add_4                       0.000126839             1.43986
call_function  sub                         0.000114918             1.30454
call_module    embed                       9.89437e-05             1.

# TorchScript

In [395]:
def workload_fm_torchscript(dim):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(22)],dim)
  fm_model_traced_ori = fx.symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(22)],dim)
  fm_model_traced_modify = fx.symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM(fm_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  ori = torch.jit.script(fm_model_traced_ori)
  modify = torch.jit.script(fm_model_traced_modify)
  ori.eval()
  modify.eval()
  return ori,modify

# dim = 8

In [396]:
ori, modify = workload_fm_torchscript(8)

now gen workload of fm with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [423]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

939 µs ± 150 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [424]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

915 µs ± 130 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [427]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.13 ms ± 197 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [428]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.3 ms ± 220 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 32

In [429]:
ori, modify = workload_fm_torchscript(32)

now gen workload of fm with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [430]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

The slowest run took 6.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1.16 ms ± 458 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [431]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

The slowest run took 6.67 times longer than the fastest. This could mean that an intermediate result is being cached.
1.17 ms ± 465 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [432]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

The slowest run took 4.32 times longer than the fastest. This could mean that an intermediate result is being cached.
2.3 ms ± 430 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [434]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.34 ms ± 188 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 64

In [435]:
ori, modify = workload_fm_torchscript(64)

now gen workload of fm with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [436]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

The slowest run took 5.25 times longer than the fastest. This could mean that an intermediate result is being cached.
1.35 ms ± 406 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [438]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

1.35 ms ± 121 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [441]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.54 ms ± 252 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [442]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

2.25 ms ± 203 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 128

In [443]:
ori, modify = workload_fm_torchscript(128)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [444]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

The slowest run took 4.75 times longer than the fastest. This could mean that an intermediate result is being cached.
1.43 ms ± 351 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [445]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

The slowest run took 7.00 times longer than the fastest. This could mean that an intermediate result is being cached.
1.26 ms ± 451 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [448]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

13.9 ms ± 744 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [449]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.16 ms ± 756 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [556]:
def custom_workload_fm(dim,nums_fields,prefix):
  print(f"now gen workload of fm with config: dim: {dim}")
  fm_model_ori = fm.FactorizationMachineModel([100 for i in range(nums_fields)],dim)
  fm_model_traced_ori = fx.symbolic_trace(fm_model_ori)
  
  fm_model_modify = fm.FactorizationMachineModel([100 for i in range(nums_fields)],dim)
  fm_model_traced_modify = fx.symbolic_trace(fm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM(fm_model_traced_modify,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "linear_offsets")
  matches = subgraph_rewriter.replace_pattern_with_filters(fm_model_traced_modify, pattern, replace,[match])
  return fm_model_traced_ori,fm_model_traced_modify

# frappe

# dim = 8

In [557]:
ori,modify = custom_workload_fm(8,10,9)

now gen workload of fm with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [558]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.5016326904296875

In [559]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.4051049550374349

In [560]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.018039385477702

In [561]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.9748379389444987

# dim = 32

In [562]:
ori,modify = custom_workload_fm(32,10,9)

now gen workload of fm with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [563]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.8848110834757487

In [564]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.7660388946533203

In [565]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.2710412343343098

In [566]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.1697689692179363

# dim = 64

In [578]:
ori,modify = custom_workload_fm(64,10,9)

now gen workload of fm with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [579]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.018699010213216

In [580]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

0.9664535522460938

In [583]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.1101881663004558

In [584]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.2107690175374348

# dim = 128

In [573]:
ori,modify = custom_workload_fm(128,10,9)

now gen workload of fm with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [574]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.2017488479614258

In [575]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.0515451431274414

In [576]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.592429478963216

In [577]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

1.386117935180664

# custom 

In [593]:
ori,modify = custom_workload_fm(32,100,50)

now gen workload of fm with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [595]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

15.770626068115234

In [597]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

8.28556219736735